# Imports

In [1]:
import pandas as pd

# Collect Data

I couldn't import the data as directly downloaded from semeval because it gave me an "unknown not utf-8 character" so I imported it as a csv to an Excel and saved it again as train_utf.csv and it worked

In [2]:
traindata = pd.read_csv("train_utf.csv", sep=',', encoding="latin1").fillna(method="ffill")
testdata = pd.read_csv("test_utf.csv", sep=',', encoding="latin1").fillna(method="ffill")

## Keep only the abortion tweets
is_abortion = traindata['Target']=="Legalization of Abortion"
is_abortion_test = testdata['Target']=="Legalization of Abortion"
train_abortion = traindata[is_abortion]
test_abortion = testdata[is_abortion_test]

# Preprocessing

In [3]:
from nltk.tokenize import TweetTokenizer
import re
import unidecode

# TweetTokenizer basically unterstands arrows, smiley faces and weird punctuation
tokenizer = TweetTokenizer(preserve_case=True, reduce_len=True, strip_handles=False)


def my_preprocess(text, keep_hashtags=True):
    toks = tokenizer.tokenize(text)

    ret = []
    for tok in toks:
        if tok[0] == "#" and not keep_hashtags:
            continue
        if tok[:4] == "http":
            continue
        if tok[0] == "@":
            continue
        # removing numbers and punctuation
        if not tok.isalpha():
            continue
        ret.append(unidecode.unidecode(tok.lower()))
    return " ".join(ret)

In [4]:

## Train
train_abortion['Text'] = train_abortion['Tweet'].apply(lambda x: my_preprocess(x))

# Shifts the order on the original tweet list
sample_train = train_abortion.sample(frac=1.0)

text_train, label_train = sample_train['Text'], sample_train['Stance']

## Test
test_abortion['Text'] = test_abortion['Tweet'].apply(lambda x: my_preprocess(x))

sample_test = test_abortion.sample(frac=1.0)

text_test, label_test = sample_test['Text'], sample_test['Stance']

/home/damifur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/damifur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


# Vectorize

Using a word count Matrix

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords

vectorizer = CountVectorizer(
    binary=True, min_df=0.002, max_df=0.55, ngram_range=(1, 4),
    #stop_words=stopwords.words('spanish')
)

X = vectorizer.fit_transform(pd.concat([text_train,text_test]))
X_train = X[:653]
X_test = X[653:]

# Simple plain old Logistic Regression

In [6]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(penalty='l1', class_weight='balanced', C=1.0, n_jobs=-1)

clf.fit(X_train, label_train)

clf.score(X_train, label_train)

/home/damifur/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/damifur/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/damifur/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


0.891271056661562

In [7]:
clf.score(X_test, label_test)

0.525

In [11]:
test_preds = clf.predict(X_test)
list(zip(text_test, test_preds, label_test))[:10]

[('i get several requests with petitions to save whales but never to stop the slaughter of babies in the wombs of mothers',
  'AGAINST',
  'AGAINST'),
 ('bullshit multiplied by bullshit is still bullshit', 'NONE', 'AGAINST'),
 ('the point is bodily autonomy a born child can rely on other people and can live by itself',
  'FAVOR',
  'AGAINST'),
 ('the most horrific violence in america is the war on babies',
  'AGAINST',
  'AGAINST'),
 ('people with a uterus can be pro choice people who have been pregnant can be pro choice people who lost a child can be prochoice',
  'FAVOR',
  'FAVOR'),
 ('those who deny women been raped abortion are the same ppl who tell rape victims they asked for it',
  'AGAINST',
  'AGAINST'),
 ('and accepting with love any children arising from union of marriage or at least taking responsibility for life created',
  'AGAINST',
  'AGAINST'),
 ('but of course today one have to be responsible you can kill your child instead',
  'AGAINST',
  'AGAINST'),
 ('or these dum

In [15]:
from sklearn.metrics import precision_score, recall_score, f1_score

f1_score(label_test, test_preds, average=None, labels=['AGAINST','FAVOR','NONE'])

# array([0.63157895, 0.41509434, 0.38666667])
# Micro 0.525 Macro 0.47777998455257636

0.47777998455257636

In [77]:
import fasttext

TypeError: train() missing 2 required positional arguments: 'ft_path' and 'corpus_file'